<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recursive_Feature_Elimination" data-toc-modified-id="Recursive_Feature_Elimination-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Recursive_Feature_Elimination</a></span></li><li><span><a href="#Recursive-Feature-Elimination" data-toc-modified-id="Recursive-Feature-Elimination-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Recursive Feature Elimination</a></span><ul class="toc-item"><li><span><a href="#train_test_split-Section" data-toc-modified-id="train_test_split-Section-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>train_test_split Section</a></span></li><li><span><a href="#NOTE-1:" data-toc-modified-id="NOTE-1:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NOTE 1:</a></span></li><li><span><a href="#RFE-fitting" data-toc-modified-id="RFE-fitting-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RFE fitting</a></span></li><li><span><a href="#NOTE-2:-Inference" data-toc-modified-id="NOTE-2:-Inference-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>NOTE 2: Inference</a></span></li></ul></li></ul></div>

# Life_Expectancy_WHO_UN_Analysis_Modeling

## Recursive_Feature_Elimination

To:&nbsp;&nbsp;&nbsp;&nbsp; [Magnimind](https://magnimindacademy.com/)

From: Matt Curcio, matt.curcio.ri@gmail.com

Date: 2023-01-29

Re:&nbsp;&nbsp;&nbsp; NOTEBOOK #6

---


- **Use** `Clean_LE_Data_FEng_4.csv`

<!-- Idea derived from
https://www.kaggle.com/code/goyalshalini93/car-price-prediction-linear-regression-rfe
-->

In [1]:
# Common Python Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# import warnings
import warnings
warnings.filterwarnings("ignore")

# Libraries from Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm 

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Reality check ;)
path = '/home/mcc/Desktop/Life_Expectancy_Analysis_Modeling/data/processed/'
fileName = 'Clean_LE_Data_FEng_4.csv'

df = pd.read_csv(path+fileName, header=0)

# Convert 4 vars to categorical
df['Country'] = pd.Categorical(df['Country'])
df['Year'] = pd.Categorical(df['Year'])
df['Status'] = pd.Categorical(df['Status'])
df['Region'] = pd.Categorical(df['Region'])

df.head(3)

,Country,Year,Status,LifeExpectancy,AdultMort,InfD,EtOH,PercExpen,Measles,BMI,lt5yD,Polio,TotalExpen,DTP,HIV,Thin1_19y,Thin5_9y,Income,Education,Region
0,Afghanistan,2015,0,65.0,263.0,62,0.01,71.279624,1154,19.1,83,6.0,8.16,65.0,0.1,17.2,17.3,0.479,10.1,2
1,Afghanistan,2014,0,59.9,271.0,64,0.01,73.523582,492,18.6,86,58.0,8.18,62.0,0.1,17.5,17.5,0.476,10.0,2
2,Afghanistan,2013,0,59.9,268.0,66,0.01,73.219243,430,18.1,89,62.0,8.13,64.0,0.1,17.7,17.7,0.470,9.9,2


## Recursive Feature Elimination

### train_test_split Section

In [3]:
x = df.drop(['LifeExpectancy','Country'], axis=1)
y = df['LifeExpectancy']

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.3,
                                                    random_state=100)

print('\nShape of x_train is', {x_train.shape})
print('\nShape of y_train is', {y_train.shape})

print('\nShape of x_test is', {x_test.shape})
print('\nShape of y_test is', {y_test.shape})


Shape of x_train is {(2049, 18)}

Shape of y_train is {(2049,)}

Shape of x_test is {(879, 18)}

Shape of y_test is {(879,)}


### NOTE 1:
- Cannot Use `Stratify`, option for `train_test_split`. There are not sufficient class labels of one of your classes to keep the data splitting ratio equal to test_size.

In [4]:
x_train

,Year,Status,AdultMort,InfD,EtOH,PercExpen,Measles,BMI,lt5yD,Polio,TotalExpen,DTP,HIV,Thin1_19y,Thin5_9y,Income,Education,Region
1346,2013,0,28.0,5,6.48,26.407266,73,51.4,6,98.0,4.30,98.0,0.1,2.4,2.5,0.782,15.0,1
2073,2006,0,83.0,0,1.28,448.595299,144,65.0,0,95.0,2.58,96.0,0.1,4.9,4.5,0.835,13.7,2
746,2005,1,92.0,0,11.28,7627.412444,2,55.0,0,93.0,9.77,93.0,0.1,1.2,1.0,0.897,16.6,5
2667,2004,0,15.0,3,1.36,379.765905,1,51.4,4,97.0,5.63,97.0,0.1,6.4,6.3,0.673,13.6,2
348,2003,0,693.0,2,5.51,299.367125,59,31.6,4,96.0,4.65,96.0,31.9,1.9,1.8,0.567,11.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2005,0,118.0,1,0.63,854.011576,25,49.5,1,99.0,2.59,99.0,0.1,7.0,6.8,0.742,11.5,2
79,2000,0,156.0,0,7.27,1127.743470,0,38.2,0,96.0,4.13,95.0,0.1,3.7,3.6,0.000,0.0,8
1859,2012,0,157.0,2,3.63,334.817425,0,51.7,3,99.0,8.40,98.0,0.1,1.8,1.7,0.625,11.5,8
2840,2007,0,157.0,0,1.01,342.490856,0,46.6,0,67.0,3.67,67.0,0.1,1.6,1.5,0.579,10.7,6


In [5]:
y_train

1346    69.5
2073    76.6
746     78.1
2667    74.0
348     46.4
        ... 
1930    74.3
79      73.6
1859    73.9
2840    73.0
1544    79.7
Name: LifeExpectancy, Length: 2049, dtype: float64

In [6]:
x_train.to_csv('x_train.csv', index=False)

x_test.to_csv('x_test.csv', index=False)

y_train.to_csv('y_train.csv', index=False)

y_test.to_csv('y_test.csv', index=False)

In [7]:
scaler = MinMaxScaler()

numerical_vars = ['AdultMort','EtOH','PercExpen',
                  'Measles','BMI','lt5yD','Polio','TotalExpen',
                  'DTP','HIV','Thin1_19y','Income','Education']
                  
x_train[numerical_vars] = scaler.fit_transform(x_train[numerical_vars])


In [8]:
x_train

,Year,Status,AdultMort,InfD,EtOH,PercExpen,Measles,BMI,lt5yD,Polio,TotalExpen,DTP,HIV,Thin1_19y,Thin5_9y,Income,Education,Region
1346,2013,0,0.037396,5,0.362262,0.001356,0.000400,0.657963,0.0024,0.989583,0.211151,0.989691,0.000000,0.083333,2.5,0.827513,0.724638,1
2073,2006,0,0.113573,0,0.071109,0.023029,0.000789,0.835509,0.0000,0.958333,0.109134,0.969072,0.000000,0.173913,4.5,0.883598,0.661836,2
746,2005,1,0.126039,0,0.631019,0.391553,0.000011,0.704961,0.0000,0.937500,0.535587,0.938144,0.000000,0.039855,1.0,0.949206,0.801932,5
2667,2004,0,0.019391,3,0.075588,0.019495,0.000005,0.657963,0.0016,0.979167,0.290036,0.979381,0.000000,0.228261,6.3,0.712169,0.657005,2
348,2003,0,0.958449,2,0.307951,0.015368,0.000323,0.399478,0.0016,0.968750,0.231910,0.969072,0.629703,0.065217,1.8,0.600000,0.570048,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2005,0,0.162050,1,0.034714,0.043841,0.000137,0.633159,0.0004,1.000000,0.109727,1.000000,0.000000,0.250000,6.8,0.785185,0.555556,2
79,2000,0,0.214681,0,0.406495,0.057893,0.000000,0.485640,0.0000,0.968750,0.201068,0.958763,0.000000,0.130435,3.6,0.000000,0.000000,8
1859,2012,0,0.216066,2,0.202688,0.017188,0.000000,0.661880,0.0012,1.000000,0.454330,0.989691,0.000000,0.061594,1.7,0.661376,0.555556,8
2840,2007,0,0.216066,0,0.055991,0.017582,0.000000,0.595300,0.0000,0.666667,0.173784,0.670103,0.000000,0.054348,1.5,0.612698,0.516908,6


### RFE fitting

In [9]:
lm = LinearRegression()
lm.fit(x_train,y_train)

LinearRegression()

In [10]:
rfe = RFE(lm)

rfe = rfe.fit(x_train, y_train)

In [11]:
feature_importance = list(zip(x_train.columns,rfe.support_,rfe.ranking_))

In [12]:
def Sort_Tuple(tup):
    """ reverse = None (Sorts in Ascending order)
    key is set to sort using second element of
    sublist lambda has been used
    """
    tup.sort(key = lambda x: x[2])
    return tup
 

# printing the sorted list of tuples
Sort_Tuple(feature_importance)

[('AdultMort', True, 1),
 ('PercExpen', True, 1),
 ('Measles', True, 1),
 ('BMI', True, 1),
 ('Polio', True, 1),
 ('DTP', True, 1),
 ('HIV', True, 1),
 ('Income', True, 1),
 ('Education', True, 1),
 ('Thin1_19y', False, 2),
 ('Status', False, 3),
 ('lt5yD', False, 4),
 ('TotalExpen', False, 5),
 ('EtOH', False, 6),
 ('Region', False, 7),
 ('InfD', False, 8),
 ('Thin5_9y', False, 9),
 ('Year', False, 10)]

### NOTE 2: Inference
    
- **USE** For first model: `Income`, `Education`,`HIV`, `DTP`, `Polio`, `lt5y_D`, `AdultMort`